In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🤖 WhatsApp Chat Style Analyzer\n",
    "\n",
    "Este notebook analiza un chat de WhatsApp y genera visualizaciones y estadísticas.\n",
    "\n",
    "## 📝 Antes de empezar:\n",
    "1. Exporta un chat de WhatsApp (sin medios)\n",
    "2. Pon el archivo .txt en esta carpeta"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📚 Importar lo necesario"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import re\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from wordcloud import WordCloud\n",
    "from collections import Counter\n",
    "from datetime import datetime\n",
    "import nltk\n",
    "try:\n",
    "    nltk.data.find('corpora/stopwords')\n",
    "except LookupError:\n",
    "    nltk.download('stopwords')\n",
    "from nltk.corpus import stopwords\n",
    "\n",
    "# Configuración de matplotlib\n",
    "plt.style.use('seaborn')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "# Definir conjunto global de palabras a excluir\n",
    "EXCLUDED_WORDS = {\n",
    "    'todo', 'porque', 'tiene', 'multimedia', 'omitido', \n",
    "    '<multimedia', 'omitido>', 'imagen', 'video', 'audio', \n",
    "    'sticker', 'gif', 'documento', 'eliminado', 'omitted',\n",
    "    'image', 'para', 'pero', 'este', 'esta', 'esto', 'como',\n",
    "    'cuando', 'donde', 'media', '<image', 'media', 'omitted',\n",
    "    'ahora', 'algo', 'aquí', 'así', 'aunque', 'bien', 'cada',\n",
    "    'casi', 'como', 'cual', 'debe', 'desde', 'después',\n",
    "    'dice', 'dijo', 'donde', 'entonces', 'entre', 'está',\n",
    "    'están', 'había', 'hace', 'hasta', 'hola', 'luego',\n",
    "    'mejor', 'menos', 'mismo', 'mucho', 'nada', 'otro',\n",
    "    'pues', 'quién', 'sabe', 'sido', 'sine', 'sino',\n",
    "    'sobre', 'solo', 'también', 'tanto', 'tengo', 'todas',\n",
    "    'todos', 'vamos', 'vaya', 'verdad', 'puede', 'pudo',\n",
    "    'quiere', 'sería', 'hacer', 'hecho', 'siendo', 'tenía',\n",
    "    'través', 'primera', 'según', 'ningún', 'manera', 'misma',\n",
    "    'image>', 'omitted>', 'attached:', 'image', 'attached','porq',\n",
    "    'bueno','gente','creo','cosa','siempre','claro','año','cierto',\n",
    "    'cómo','gran','toda','años','decir','dicho','tiempo','parece'\n",
    "}"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📂 Buscar archivos de chat"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Ver qué archivos de chat hay disponibles\n",
    "chat_files = [f for f in os.listdir() if f.endswith('.txt')]\n",
    "\n",
    "print(\"📁 Archivos encontrados:\")\n",
    "for i, file in enumerate(chat_files, 1):\n",
    "    print(f\"{i}. {file}\")\n",
    "\n",
    "if not chat_files:\n",
    "    print(\"❌ No se encontraron archivos .txt\")\n",
    "    raise Exception(\"Necesitas exportar un chat de WhatsApp primero\")\n",
    "\n",
    "# Elegir un archivo\n",
    "file_number = int(input(\"\\nElige el número del archivo a analizar: \"))\n",
    "selected_file = chat_files[file_number - 1]\n",
    "print(f\"Archivo seleccionado: {selected_file}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📖 Leer y procesar el chat"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Leer el archivo y mostrar las primeras líneas para debug\n",
    "with open(selected_file, 'r', encoding='utf-8') as file:\n",
    "    content = file.read()\n",
    "    \n",
    "print(\"Primeras 5 líneas del archivo:\")\n",
    "print(\"\\n\".join(content.split('\\n')[:5]))\n",
    "print(\"\\nTotal de líneas:\", len(content.split('\\n')))\n",
    "\n",
    "# Intentar diferentes patrones de regex para WhatsApp\n",
    "patterns = [\n",
    "    # Patrón 1: Formato típico de WhatsApp\n",
    "    r'(\\d{1,2}/\\d{1,2}/\\d{2,4},\\s\\d{1,2}:\\d{2})\\s-\\s([^:]+):\\s(.+)',\n",
    "    \n",
    "    # Patrón 2: Con AM/PM\n",
    "    r'(\\d{1,2}/\\d{1,2}/\\d{2,4},\\s\\d{1,2}:\\d{2}\\s[APMapm]{2})\\s-\\s([^:]+):\\s(.+)',\n",
    "    \n",
    "    # Patrón 3: Formato iPhone\n",
    "    r'\\[(\\d{1,2}/\\d{1,2}/\\d{2},\\s\\d{1,2}:\\d{2}(?::\\d{2})?)\\]\\s([^:]+):\\s(.+)'\n",
    "]\n",
    "\n",
    "matches = []\n",
    "used_pattern = None\n",
    "\n",
    "for pattern in patterns:\n",
    "    matches = re.findall(pattern, content, re.MULTILINE)\n",
    "    if matches:\n",
    "        used_pattern = pattern\n",
    "        print(f\"\\n✅ Patrón encontrado: {pattern}\")\n",
    "        print(f\"Mensajes encontrados: {len(matches)}\")\n",
    "        break\n",
    "\n",
    "if not matches:\n",
    "    print(\"\\n❌ No se pudo encontrar un patrón válido en el archivo\")\n",
    "    print(\"Por favor, verifica que el archivo es una exportación de WhatsApp\")\n",
    "    raise Exception(\"Formato de archivo no reconocido\")\n",
    "\n",
    "# Crear DataFrame\n",
    "df = pd.DataFrame(matches, columns=['datetime', 'sender', 'message'])\n",
    "print(\"\\nEstructura del DataFrame:\")\n",
    "print(df.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Procesamiento avanzado"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Procesar fechas y añadir columnas adicionales\n",
    "df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%m/%y, %H:%M')\n",
    "df['hour'] = df['datetime'].dt.hour\n",
    "df['day_of_week'] = df['datetime'].dt.day_name()\n",
    "\n",
    "# Identificar tipo de mensaje\n",
    "df['type'] = df['message'].apply(lambda x: 'media' if '<Multimedia omitido>' in x or 'Media omitted' in x else 'text')\n",
    "\n",
    "# Función para filtrar palabras\n",
    "def filter_words(messages):\n",
    "    \"\"\"Filtra las palabras de una lista de mensajes aplicando todos los criterios de exclusión.\"\"\"\n",
    "    stop_words = set(stopwords.words('spanish'))\n",
    "    stop_words.update(EXCLUDED_WORDS)\n",
    "    \n",
    "    words = []\n",
    "    for message in messages:\n",
    "        message_words = message.lower().split()\n",
    "        filtered_words = [word for word in message_words \n",
    "                        if word not in stop_words\n",
    "                        and len(word) > 3 \n",
    "                        and not word.startswith('http')\n",
    "                        and not word.startswith('<')\n",
    "                        and not word.endswith('>')\n",
    "                        and not any(char.isdigit() for char in word)]\n",
    "        words.extend(filtered_words)\n",
    "    \n",
    "    return words"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Visualizaciones generales"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Mensajes por persona\n",
    "plt.figure(figsize=(12, 6))\n",
    "messages_per_person = df['sender'].value_counts().head(20)\n",
    "messages_per_person.plot(kind='bar', color='skyblue')\n",
    "plt.title('Top 20: Mensajes por persona')\n",
    "plt.xlabel('Miembro')\n",
    "plt.ylabel('Número de mensajes')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# 2. Actividad a lo largo del tiempo\n",
    "plt.figure(figsize=(12, 6))\n",
    "daily_activity = df.resample('D', on='datetime').size()\n",
    "daily_activity.plot(kind='line', color='green')\n",
    "plt.title('Actividad diaria del chat')\n",
    "plt.xlabel('Fecha')\n",
    "plt.ylabel('Número de mensajes')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# 3. Distribución de tipos de mensajes\n",
    "plt.figure(figsize=(8, 8))\n",
    "type_counts = df['type'].value_counts()\n",
    "type_counts.plot(kind='pie', autopct='%1.1f%%', colors=['lightcoral', 'lightblue'])\n",
    "plt.title('Distribución de tipos de mensajes')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# 4. Actividad por hora\n",
    "plt.figure(figsize=(12, 6))\n",
    "df['hour'].value_counts().sort_index().plot(kind='bar')\n",
    "plt.title('Actividad por hora del día')\n",
    "plt.xlabel('Hora')\n",
    "plt.ylabel('Número de mensajes')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# 5. Actividad por día de la semana\n",
    "plt.figure(figsize=(12, 6))\n",
    "day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']\n",
    "df['day_of_week'].value_counts().reindex(day_order).plot(kind='bar')\n",
    "plt.title('Actividad por día de la semana')\n",
    "plt.xlabel('Día')\n",
    "plt.ylabel('Número de mensajes')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# 6. Nube de palabras general del grupo\n",
    "text_messages = df[df['type'] == 'text']['message']\n",
    "words = filter_words(text_messages)\n",
    "text = ' '.join(words)\n",
    "\n",
    "stop_words = set(stopwords.words('spanish'))\n",
    "stop_words.update(EXCLUDED_WORDS)\n",
    "\n",
    "wordcloud = WordCloud(\n",
    "    width=1200, \n",
    "    height=600, \n",
    "    background_color='white',\n",
    "    max_words=150,\n",
    "    collocations=False,\n",
    "    stopwords=stop_words\n",
    ").generate(text)\n",
    "\n",
    "plt.figure(figsize=(15, 7.5))\n",
    "plt.imshow(wordcloud, interpolation='bilinear')\n",
    "plt.axis('off')\n",
    "plt.title('Palabras más usadas en el grupo')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 👥 Análisis por miembro"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Ver miembros del chat\n",
    "members = df['sender'].unique().tolist()\n",
    "\n",
    "print(f\"\\n👥 Miembros encontrados ({len(members)}):\")\n",
    "for i, member in enumerate(members, 1):\n",
    "    print(f\"{i}. {member}\")\n",
    "\n",
    "if not members:\n",
    "    print(\"❌ No se encontraron miembros en el chat\")\n",
    "    raise Exception(\"No se pudieron extraer los miembros del chat\")\n",
    "\n",
    "# Seleccionar un miembro para analizar\n",
    "member_number = int(input(\"\\nElige el número del miembro a analizar: \"))\n",
    "selected_member = members[member_number - 1]\n",
    "\n",
    "# Mostrar estadísticas del miembro\n",
    "member_messages = df[df['sender'] == selected_member]\n",
    "member_count = len(member_messages)\n",
    "total_messages = len(df)\n",
    "percentage = round(member_count / total_messages * 100, 1)\n",
    "\n",
    "print(f\"\\n📊 Estadísticas de {selected_member}:\")\n",
    "print(f\"📝 Mensajes: {member_count}\")\n",
    "print(f\"📊 Porcentaje: {percentage}%\")\n",
    "\n",
    "# Obtener palabras más frecuentes\n",
    "text_messages = member_messages[member_messages['type'] == 'text']['message'].tolist()\n",
    "words = filter_words(text_messages)\n",
    "word_freq = Counter(words)\n",
    "print(\"\\n🔤 Palabras más usadas:\")\n",
    "for word, count in word_freq.most_common(5):\n",
    "    print(f\"- {word}: {count} veces\")\n",
    "\n",
    "# Generar nube de palabras del miembro\n",
    "text = ' '.join(words)\n",
    "wordcloud = WordCloud(\n",
    "    width=800, \n",
    "    height=400, \n",
    "    background_color='white',\n",
    "    max_words=100,\n",
    "    collocations=False,\n",
    "    stopwords=stop_words\n",
    ").generate(text)\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "plt.imshow(wordcloud, interpolation='bilinear')\n",
    "plt.axis('off')\n",
    "plt.title(f'Palabras más usadas por {selected_member}')\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}